# Neighborhood-Based Collaborative Filtering

One basis for recommendation algorithms is collaborative filtering, where we use the historical rating behaviors to find similarities across users and/or items in arriving at the rating predictions.  Here, we explore a simple, yet highly intuitive collaborative filtering approach that is based on the concept of neighborhood.  Not only does this approach clearly illustrate the collaborative filtering notion of bridging users and items by finding similarities, but it also lends itself to interpretability as we will see in this tutorial.

추천알고리즘의 가장 기본은 협업 필터링입니다. 평점 예측을 위해서 유저 또는 아이템간의 과거의 평점을 기반한 유사도를 찾습니다. 여기서 이웃에 기반한 간단하지만 매우 직관적인 협업 필터링 접근법을 알아보겠습니다. 이 튜토리얼에서는 유저와 아이템들의 유사성을 나타내는 협업 필터링의 방식 뿐만 아니라 그 자신 또한 해석할 수 있음을 이끕니다.

## 1. Setup

In [2]:
#!pip install --quiet cornac==1.5.1

In [3]:
import os
import sys
import textwrap
from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import cornac
from cornac.utils import cache
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.models import UserKNN, ItemKNN

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

SEED = 42
VERBOSE = False

System version: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
Cornac version: 1.12.0


In [4]:
sample_df = pd.read_csv(
  cache("https://static.preferred.ai/tutorials/recommender-systems/sample_data.csv"), 
  sep=",", names=["UserID", "ItemID", "Rating"]
)
sample_df.head()

,UserID,ItemID,Rating
0,1,1,7
1,1,2,6
2,1,3,7
3,1,4,4
4,1,5,5


### Original Rating Matrix

Let us see this sample dataset in matrix form.


간단하게 행렬의 형태로 샘플 데이터셋을 알아봅시다

In [5]:
dataset = cornac.data.Dataset.from_uir(sample_df.itertuples(index=False))
df = pd.DataFrame(dataset.matrix.A,
                  index=[f"User {u + 1}" for u in np.arange(dataset.num_users)],
                  columns=[f"Item {i + 1}" for i in np.arange(dataset.num_items)])
df.replace(0, np.nan, inplace=True)
df["Mean Rating"] = df.mean(axis=1)
df

,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Mean Rating
User 1,7.0,6.0,7.0,4.0,5.0,4.0,5.5
User 2,6.0,7.0,NaN,4.0,3.0,4.0,4.8
User 3,NaN,3.0,3.0,1.0,1.0,NaN,2.0
User 4,1.0,2.0,2.0,3.0,3.0,4.0,2.5
User 5,1.0,NaN,1.0,2.0,3.0,3.0,2.0


### Mean-Centered Rating Matrix

One concern about rating data is its subjectivity.  In particular, different users may use different ranges.  Some users are lenient and tend to assign higher ratings.  Others are strict and tend to assign lower ratings.  A commonly adopted approach to 'normalize' the ratings is to take the mean of the ratings by a user and subtract the mean from the individual ratings of the said user.

평점 데이터의 한가지 걱정은 주관성입니다. 특히 많은 유저들은 서로 다른 범위를 사용합니다. 몇몇 유저들은 관대해서 높은 점수를 주는 경향이 있습니다. 또 다른 사람들은 매우 딱딱하고 낮은 점수를 줍니다. 일반적으로 사용되는 방법인 '정규화' 는 유저들이 만들어낸 평점에서 평균을 취하고 각 유저의 평점에서 평균을 빼는 방식을 사용합니다.

## 3. User-Based Methods

For each user, her mean rating is calculated as follows:  

각 유저마다, 평균 평점은 다음과 같이 계산됩니다.

$$ \mu_u = \frac{\Sigma_{k \in \mathcal{I}_u} r_{uk}}{|\mathcal{I}_u|} \ \ \forall u \in \{1 \dots m\} $$


Two common approaches to measure similarity between two users $\mathrm{Sim}(u, v)$ are *Cosine similarity* and *Pearson correlation coefficient*:  

두 유저의 유사도 $\mathrm{Sim}(u, v)$를 계산하는 일반적인 접근 방법은 *코사인 유사도* 와 *피어슨 상관계수*가 있습니다.

\begin{align*}
\mathrm{Cosine}(u,v) &= \frac{\Sigma_{k \in \mathcal{I}_u \cap \mathcal{I}_v} r_{uk} * r_{vk}}{\sqrt{\Sigma_{k \in \mathcal{I}_u \cap \mathcal{I}_v} r_{uk}^2} * \sqrt{\Sigma_{k \in \mathcal{I}_u \cap \mathcal{I}_v} r_{vk}^2}} \\
\mathrm{Pearson}(u,v) &= \frac{\Sigma_{k \in \mathcal{I}_u \cap \mathcal{I}_v} (r_{uk} - \mu_u) * (r_{vk} - \mu_v)}{\sqrt{\Sigma_{k \in \mathcal{I}_u \cap \mathcal{I}_v} (r_{uk} - \mu_u)^2} * \sqrt{\Sigma_{k \in \mathcal{I}_u \cap \mathcal{I}_v} (r_{vk} - \mu_v)^2}}
\end{align*}


For example, given the original rating matrix, between *User 1* and *User 3* we have their similarities as:  
예를들어, 원래의 평점 행렬에서 *User 1*과 *User 3*의 유사도는 다음과 같습니다

\begin{align*}
\mathrm{Cosine}(1,3) &= \frac{6*3+7*3+4*1+5*1}{\sqrt{6^2+7^2+4^2+5^2} * \sqrt{3^2+3^2+1^2+1^2}} = 0.956 \\
\mathrm{Pearson}(1,3) &= \frac{(6 - 5.5) * (3 - 2) + (7 - 5.5) * (3 - 2) + (4 - 5.5) * (1 - 2) + (5 - 5.5) * (1 - 2)}{\sqrt{0.5^2 + 1.5^2 + (-1.5)^2 + (-0.5)^2} * \sqrt{1^2 + 1^2 + (-1)^2 + (-1)^2}} = 0.894
\end{align*}


Let's validate the calculation:  
이 계산을 검증해봅시다

In [6]:
uknn_cosine = UserKNN(k=2, similarity="cosine", verbose=VERBOSE).fit(dataset)
print(f"Cosine(1,3) = {uknn_cosine.sim_mat[0, 2]:.3f}")

uknn_pearson = UserKNN(k=2, similarity="pearson", verbose=VERBOSE).fit(dataset)
print(f"Pearson(1,3) = {uknn_pearson.sim_mat[0, 2]:.3f}")

Cosine(1,3) = 0.956
Pearson(1,3) = 0.894


Here we use the weighted average of the mean-centered rating to provide prediction.  The overall neighborhood-based *prediction function* is as follows:  

여기서는 우리가 가중치평균을 가진 평균 중심 평점을 예측하는데 사용합니다. 전체적인 이웃-기반 예측 함수는 다음과 같습니다.

$$ \hat{r}_{uj} = \mu_u + \frac{\Sigma_{v \in P_u(j)} \mathrm{Sim}(u,v) * (r_{vj} - \mu_v)}{\Sigma_{v \in P_u(j)} |\mathrm{Sim}(u,v)|} $$


For example, to calculate the predicted rating given by *User 3* to *Item 1* and *Item 6*, where the ratings are based on the two nearest neighbors (*User 1* and *User 2*):

예를들면 *Item 1* 과 *Item 6* 에 관해서 두개의 가장 가까운 이웃(*User 1* 과 *User 2*)에 기반해서 주어진 *User 3*에 대해서 예측을 하기 위해 계산하면, 

\begin{align*}
\hat{r}_{31} &= 2 + \frac{1.5*0.894+1.2*0.939}{0.894 + 0.939} = 3.35 \\
\hat{r}_{36} &= 2 + \frac{-1.5*0.894-0.8*0.939}{0.894 + 0.939} = 0.86
\end{align*}


Let's validate the calculation:  
이 계산을 검증해봅시다

In [7]:
print(f"R(3,1) = {uknn_pearson.score(user_idx=2, item_idx=0):.2f}")
print(f"R(3,6) = {uknn_pearson.score(user_idx=2, item_idx=5):.2f}")

R(3,1) = 3.35
R(3,6) = 0.86


### Let's Try It on a Real Dataset

We will now employ these algorithms on the [MovieLens](https://grouplens.org/datasets/movielens/) dataset. Our convention is to use the mean-centered ratings when aggregating the neighbors' ratings to produce a prediction and thereafter adjusting for the mean.  However, we can base the similarity computation based on either the original ratings or the mean-centered ratings, as shown in the variations below.

우리가 이제 이 알고리즘들 MovieLens 데이터셋에 적용시킬겁니다. 우리의 목적은 이웃의 평점을 다시 집계할때 평균 중심 평점을 사용하여 예측을 생성한후 평균에 맞게 조정하는것입니다. 하지만 우리는 아래의 변형에서 볼 수 있으시 원래 평점 또는 평균 중심 평점을 기반한 유사도계산을 할 수 있다.

In [8]:
# UserKNN methods
K = 50  # number of nearest neighbors
uknn_cosine = UserKNN(
  k=K, similarity="cosine", name="UserKNN-Cosine", verbose=VERBOSE
)
uknn_cosine_mc = UserKNN(
  k=K, similarity="cosine", mean_centered=True, name="UserKNN-Cosine-MC", verbose=VERBOSE
)
uknn_pearson = UserKNN(
  k=K, similarity="pearson", name="UserKNN-Pearson", verbose=VERBOSE
)
uknn_pearson_mc = UserKNN(
  k=K, similarity="pearson", mean_centered=True, name="UserKNN-Pearson-MC", verbose=VERBOSE
)

feedback = movielens.load_feedback(variant="100K")
ratio_split = RatioSplit(feedback, test_size=0.1, seed=SEED, verbose=VERBOSE)
cornac.Experiment(
  eval_method=ratio_split,
  models=[uknn_cosine, uknn_cosine_mc, uknn_pearson, uknn_pearson_mc],
  metrics=[cornac.metrics.RMSE()],
).run()


TEST:
...
                   |   RMSE | Train (s) | Test (s)
------------------ + ------ + --------- + --------
UserKNN-Cosine     | 0.9102 |    0.1076 |   1.1390
UserKNN-Cosine-MC  | 0.8947 |    0.0884 |   1.1280
UserKNN-Pearson    | 0.8947 |    0.1450 |   0.9777
UserKNN-Pearson-MC | 0.8947 |    0.0846 |   1.0759

